# Import

In [1]:
import pandas as pd

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

In [17]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # 1번 GPU만 노출됨

# PDF 배분

In [ ]:
import os
import shutil

def distribute_pdfs(source_dir, target_base_dir, target_folders):
    # PDF 파일 목록 가져오기
    pdf_files = [f for f in os.listdir(source_dir) if f.endswith(".pdf")]
    pdf_files.sort()  # 정렬하여 일관된 배분 유지
    
    # 대상 폴더 생성
    target_dirs = [os.path.join(target_base_dir, folder) for folder in target_folders]
    for directory in target_dirs:
        os.makedirs(directory, exist_ok=True)
    
    # 파일을 순차적으로 배분 (정확한 개수 유지)
    for i, pdf in enumerate(pdf_files):
        target_folder = target_dirs[i % len(target_folders)]  # 순환할당
        source_path = os.path.join(source_dir, pdf)
        target_path = os.path.join(target_folder, pdf)
        shutil.copy2(source_path, target_path)  # 복사 방식 사용
        print(f"Copied {pdf} -> {target_folder}")

# 설정
source_directory = "/home/wanted-1/potenup-workspace/Project/dacon/DACON-construction-accident-prevention/data/pdf"
target_base_directory = "/home/wanted-1/potenup-workspace/Project/dacon/DACON-construction-accident-prevention/code"
target_folders = ["JaeSik", "JiMin", "JinGyu", "SangGyeom"]

distribute_pdfs(source_directory, target_base_directory, target_folders)

# Data Load & Pre-processing

In [2]:
train = pd.read_csv('../../data/ref_train.csv', encoding = 'utf-8-sig')
test = pd.read_csv('../../data/test.csv', encoding = 'utf-8-sig')

In [3]:
# 데이터 전처리
train['공사종류(대분류)'] = train['공사종류'].str.split(' / ').str[0]
train['공사종류(중분류)'] = train['공사종류'].str.split(' / ').str[1]
train['공종(대분류)'] = train['공종'].str.split(' > ').str[0]
train['공종(중분류)'] = train['공종'].str.split(' > ').str[1]
train['사고객체(대분류)'] = train['사고객체'].str.split(' > ').str[0]
train['사고객체(중분류)'] = train['사고객체'].str.split(' > ').str[1]

test['공사종류(대분류)'] = test['공사종류'].str.split(' / ').str[0]
test['공사종류(중분류)'] = test['공사종류'].str.split(' / ').str[1]
test['공종(대분류)'] = test['공종'].str.split(' > ').str[0]
test['공종(중분류)'] = test['공종'].str.split(' > ').str[1]
test['사고객체(대분류)'] = test['사고객체'].str.split(' > ').str[0]
test['사고객체(중분류)'] = test['사고객체'].str.split(' > ').str[1]

In [4]:
# 훈련 데이터 통합 생성
combined_training_data = train.apply(
    lambda row: {
        "question": (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중 "
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

# DataFrame으로 변환
combined_training_data = pd.DataFrame(list(combined_training_data))

In [5]:
# 테스트 데이터 통합 생성
combined_test_data = test.apply(
    lambda row: {
        "question": (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중 "
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        )
    },
    axis=1
)

# DataFrame으로 변환
combined_test_data = pd.DataFrame(list(combined_test_data))

# Model import

In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16   
)

In [7]:
model_id = "NCSOFT/Llama-VARCO-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

# Vector store 생성

In [9]:
combined_training_data.head()

,question,answer
0,"공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류...",고소작업 시 추락 위험이 있는 부위에 안전장비 설치.
1,"공사종류 대분류 '토목', 중분류 '터널' 공사 중 공종 대분류 '토목', 중분류 ...",재발 방지 대책 마련과 안전교육 실시.
2,"공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류...",현장자재 정리와 안전관리 철저를 통한 재발 방지 대책 및 공문 발송을 통한 향후 조...
3,"공사종류 대분류 '토목', 중분류 '하천' 공사 중 공종 대분류 '토목', 중분류 ...","위험성 평가 및 교육을 통해 작업장 내 위험요인과 안전수칙을 근로자에게 전파하고, ..."
4,"공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류...",자재 정리 작업 시 세부 작업 방법에 대한 교육 실시와 작업 구간 이동 경로 점검 ...


In [ ]:
# Train 데이터 준비
train_questions_prevention = combined_training_data['question'].tolist()
train_answers_prevention = combined_training_data['answer'].tolist()

train_documents = [
    f"Q: {q1}\nA: {a1}" 
    for q1, a1 in zip(train_questions_prevention, train_answers_prevention)
]

# 임베딩 생성
embedding_model_name = "jhgan/ko-sbert-nli"  # 임베딩 모델 선택
embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)

# 벡터 스토어에 문서 추가
csv_vector_store = FAISS.from_texts(train_documents, embedding)

In [8]:
from langchain_community.document_loaders import DirectoryLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = DirectoryLoader('../../data/pdf', glob = '*.pdf', loader_cls = PyMuPDFLoader)

pdf_docs = loader.load()

# 문서 분할 (Chunking) 수행
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=256, chunk_overlap=50, length_function=len, separators = ['\n\n', '\n', '.', ' ']
)
split_pdf_docs = text_splitter.split_documents(pdf_docs)

In [9]:
len(split_pdf_docs)

5211

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
import os
import pandas as pd

### 🔹 2️⃣ PDF 데이터 처리 및 Vector Store 저장
def load_pdfs_from_folder(pdf_folder):
    """폴더 내 모든 PDF 문서를 LangChain을 통해 로드"""
    pdf_documents = []
    
    for pdf_file in os.listdir(pdf_folder):
        if pdf_file.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, pdf_file)
            loader = PyPDFLoader(pdf_path)  # LangChain PDF 로더
            pdf_documents.extend(loader.load())  # 문서 추가

    return pdf_documents

# PDF 데이터 로드
pdf_folder_path = "../../data/건설안전지침"
pdf_docs = load_pdfs_from_folder(pdf_folder_path)

# 문서 분할 (Chunking) 수행
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=256, chunk_overlap=50, length_function=len
)
split_pdf_docs = text_splitter.split_documents(pdf_docs)

# PDF 데이터를 벡터화하여 FAISS 저장소 생성
pdf_vector_store = FAISS.from_documents(split_pdf_docs, embedding)

# RAG chain 생성

In [ ]:
from langchain.schema import Document
from langchain.retrievers import EnsembleRetriever

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,  # sampling 활성화
    temperature=0.1,
    return_full_text=False,
    max_new_tokens=64,
)

prompt_template = """
### 지침: 당신은 건설 안전 전문가입니다.
질문에 대한 답변을 핵심 내용만 요약하여 간략하게 작성하세요.
- 서론, 배경 설명 또는 추가 설명을 절대 포함하지 마세요.
- 다음과 같은 조치를 취할 것을 제안합니다: 와 같은 내용을 포함하지 마세요.

{context}

### 질문:
{question}

[/INST]

"""

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# 커스텀 프롬프트 생성
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# CSV 및 PDF retriever를 결합
ensemble_retriever = EnsembleRetriever(
    retrievers=[csv_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5}), pdf_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})],
    weights=[0.5, 0.5]  # CSV와 PDF 검색 결과를 동일한 가중치로 조합
)

# RAG 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,  
    chain_type="stuff",  # 단순 컨텍스트 결합 방식 사용
    retriever=ensemble_retriever,     # 2개의 저장소에서 검색
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}  # 커스텀 프롬프트 적용
)

Device set to use cuda:0


# Inference

In [19]:
# 테스트 실행 및 결과 저장
test_results = []

print("테스트 실행 시작... 총 테스트 샘플 수:", len(combined_test_data))

for idx, row in combined_test_data.iterrows():
    # 50개당 한 번 진행 상황 출력
    if (idx + 1) % 50 == 0 or idx == 0:
        print(f"\n[샘플 {idx + 1}/{len(combined_test_data)}] 진행 중...")

    # RAG 체인 호출 및 결과 생성
    prevention_result = qa_chain.invoke(row['question'])

    # 결과 저장
    result_text = prevention_result['result']
    test_results.append(result_text)

print("\n테스트 실행 완료! 총 결과 수:", len(test_results))

테스트 실행 시작... 총 테스트 샘플 수: 964

[샘플 1/964] 진행 중...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



[샘플 50/964] 진행 중...

[샘플 100/964] 진행 중...

[샘플 150/964] 진행 중...

[샘플 200/964] 진행 중...

[샘플 250/964] 진행 중...

[샘플 300/964] 진행 중...

[샘플 350/964] 진행 중...

[샘플 400/964] 진행 중...

[샘플 450/964] 진행 중...

[샘플 500/964] 진행 중...

[샘플 550/964] 진행 중...

[샘플 600/964] 진행 중...

[샘플 650/964] 진행 중...

[샘플 700/964] 진행 중...

[샘플 750/964] 진행 중...

[샘플 800/964] 진행 중...

[샘플 850/964] 진행 중...

[샘플 900/964] 진행 중...

[샘플 950/964] 진행 중...

테스트 실행 완료! 총 결과 수: 964


# Submission

In [20]:
from sentence_transformers import SentenceTransformer

embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformer(embedding_model_name)

# 문장 리스트를 입력하여 임베딩 생성
pred_embeddings = embedding.encode(test_results)
print(pred_embeddings.shape)  # (샘플 개수, 768)

(964, 768)


In [23]:
submission = pd.read_csv('../data/sample_submission.csv', encoding = 'utf-8-sig')

# 최종 결과 저장
submission.iloc[:,1] = test_results
submission.iloc[:,2:] = pred_embeddings
submission.head()

# 최종 결과를 CSV로 저장
submission.to_csv('baseline_submission.csv', index=False, encoding='utf-8-sig')

In [24]:
submission['재발방지대책 및 향후조치계획']

0      ### 답변:\n사고 원인 분석 결과 다음과 같은 재발 방지 대책을 마련할 것을 제...
1      ### 답변:\n1. 절단 및 가공 작업에 대한 사전 안전교육 강화\n   - 작업...
2      ### 답변:\n사고 방지 및 재발 방지 대책으로는 다음과 같은 조치가 필요합니다:...
3      ### 답변:\n주어진 사고 원인에 대응하기 위해 다음과 같은 재발 방지 대책과 향...
4      ### 답변:\n사고 원인 분석 결과, 주된 원인은 점심식사를 위한 이동 시 작업자...
                             ...                        
959    ### 답변:\n안전장비 착용의 철저성 검토와 안전 교육 강화, 석재의 품질 관리 ...
960    ### 답변:\n이 사고를 방지하기 위한 재발 방지 대책과 향후 조치 계획은 다음과...
961    ### 답변:\n1. 사고 원인 분석 및 보고: 사고 발생 원인을 철저히 분석하고 ...
962    주어진 정보를 바탕으로 답변을 드리겠습니다.\n\nA: \n사고 원인 분석 결과, ...
963    ### 답변:\n1. 각도절단기 사용 시 안전모 및 방호덮개 착용의 중요성에 대한 ...
Name: 재발방지대책 및 향후조치계획, Length: 964, dtype: object